In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def gaussian1d(sigma, mean, x, ord):
    x = np.array(x)
    x_ = x - mean
    var = sigma**2

    # Gaussian Function
    g1 = (1/np.sqrt(2*np.pi*var))*(np.exp((-1*x_*x_)/(2*var)))

    if ord == 0:
        g = g1
        return g
    elif ord == 1:
        g = -g1*((x_)/(var))
        return g
    else:
        g = g1*(((x_*x_) - var)/(var**2))
        return g

def gaussian2d(sup, scales):
    var = scales * scales
    shape = (sup,sup)
    n,m = [(i - 1)/2 for i in shape]
    x,y = np.ogrid[-m:m+1,-n:n+1]
    g = (1/np.sqrt(2*np.pi*var))*np.exp( -(x*x + y*y) / (2*var) )
    return g

def log2d(sup, scales):
    var = scales * scales
    shape = (sup,sup)
    n,m = [(i - 1)/2 for i in shape]
    x,y = np.ogrid[-m:m+1,-n:n+1]
    g = (1/np.sqrt(2*np.pi*var))*np.exp( -(x*x + y*y) / (2*var) )
    h = g*((x*x + y*y) - var)/(var**2)
    return h

def makefilter(scale, phasex, phasey, pts, sup):

    gx = gaussian1d(3*scale, 0, pts[0,...], phasex)
    gy = gaussian1d(scale,   0, pts[1,...], phasey)

    image = gx*gy

    image = np.reshape(image,(sup,sup))
    return image

def makeLMfilters():
    sup     = 49
    scalex  = np.sqrt(2) * np.array([1,2,3])
    norient = 6
    nrotinv = 12

    nbar  = len(scalex)*norient
    nedge = len(scalex)*norient
    nf    = nbar+nedge+nrotinv
    F     = np.zeros([sup,sup,nf])
    hsup  = (sup - 1)/2

    x = [np.arange(-hsup,hsup+1)]
    y = [np.arange(-hsup,hsup+1)]

    [x,y] = np.meshgrid(x,y)

    orgpts = [x.flatten(), y.flatten()]
    orgpts = np.array(orgpts)

    count = 0
    for scale in range(len(scalex)):
        for orient in range(norient):
            angle = (np.pi * orient)/norient
            c = np.cos(angle)
            s = np.sin(angle)
            rotpts = [[c+0,-s+0],[s+0,c+0]]
            rotpts = np.array(rotpts)
            rotpts = np.dot(rotpts,orgpts)
            F[:,:,count] = makefilter(scalex[scale], 0, 1, rotpts, sup)
            F[:,:,count+nedge] = makefilter(scalex[scale], 0, 2, rotpts, sup)
            count = count + 1

    count = nbar+nedge
    scales = np.sqrt(2) * np.array([1,2,3,4])

    for i in range(len(scales)):
        F[:,:,count]   = gaussian2d(sup, scales[i])
        count = count + 1

    for i in range(len(scales)):
        F[:,:,count] = log2d(sup, scales[i])
        count = count + 1

    for i in range(len(scales)):
        F[:,:,count] = log2d(sup, 3*scales[i])
        count = count + 1

    return F

# Call the make filter function
F = makeLMfilters()
# print (F.shape)

In [0]:
import cv2
def read_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img

In [0]:
import numpy as np
def split_image(img):
    [m,n] = img.shape
    x = n//5
    img1 = img[:,0:x]
    img2 = img[:,x:2*x]
    img3 = img[:,2*x:3*x]
    img4 = img[:,3*x:4*x]
    img5 = img[:,4*x:5*x]
    return [img1,img2,img3,img4,img5]

In [0]:
def preprocess_image(img):
    splitted = split_image(img)
    out=[]
    for k in splitted:
        k = cv2.equalizeHist(k)
        k = cv2.resize(k,(120,120))
        k = cv2.GaussianBlur(k,(5,5),2)
        out.append(k)
    return out

In [0]:
def apply_MR_filter(im_list):
  out = []
  for img in im_list:
    k = np.zeros((120,120,48))
    for i in range(0,47):
      k[:,:,i] = cv2.filter2D(img, -1, F[:,:,i])
    out.append(k)  
  return out

In [0]:
'''
Dataset description
Images of 150 persons from different angles
Total images = 19516*5 of size 120*120*48 ====> 48 is the no. of filters
'''
dataset_directory = 'drive/My Drive/input/'
dir_list = [None]*12
dir_list[0] = dataset_directory + 'v1_note4_dark/dark/'
dir_list[1] = dataset_directory + 'v1_note4_daylight/daylight/'
dir_list[2] = dataset_directory + 'v1_note4_office/office/'
dir_list[3] = dataset_directory + 'v1_oppo_dark/dark/'
dir_list[4] = dataset_directory + 'v1_oppo daylight/daylight'
dir_list[5] = dataset_directory + 'v1_oppo_office/office'
dir_list[6] = dataset_directory + 'v2_note4_dark/dark'
dir_list[7] = dataset_directory + 'v2_note4_daylight/daylight'
dir_list[8] = dataset_directory + 'v2_note4_office/office'
dir_list[9] = dataset_directory + 'v2_oppo_dark/dark'
dir_list[10] = dataset_directory + 'v2_oppo_daylight/daylight'
dir_list[11] = dataset_directory + 'v2_oppo_office/office'
from glob import glob
all_images = []
for direc in dir_list:
    images = glob(direc+'S1/'+'*.png')
    all_images+=images
    images = glob(direc+'S2/'+'*.png')
    all_images+=images

#till here, we've got a list of names of all images
#image name: <id_of that_person>_<l/r>_<img_no.>.png

In [0]:
len(all_images)

19516

In [0]:
ids = {}
for img_name in all_images:
  _,_,_,_,_,_,x = img_name.split('/')
  a,b,c = x.split('_')
  if(a not in ids):
    ids[a]=[]
  ids[a].append(img_name)

In [0]:
ids_to_labels = []
for key in ids:
  ids_to_labels.append(int(key))

In [0]:
import numpy as np
X = np.zeros((19516*5,120*120*48),dtype='uint8')
Y = np.zeros((19516*5,150),dtype='bool')
total = 0
for key in ids:
  ind = ids_to_labels.index(int(key))
  img = ids[key][0]
  x = read_image(img)
  im_list = preprocess_image(x)
  out = apply_MR_filter(im_list)
  for imm in out:
    X[total,:] = imm.reshape((120*120*48))
    Y[total,ind] = 1 
    total+=1

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [0]:
# Importing modules to create our layers and model.
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

Using TensorFlow backend.


In [0]:
#Hyperparameters
#You can change their values
lambda_ = .01
epochs = 20
batch_size = 1000

In [0]:
#corresponding to one image in dataset, there are 5 images that need to be separated. and for each of them, 
#there will be 48 images of their features. so 5*48 images will be there for one image in dataset.

encoding_dim = 36
input_img = Input(shape=(120*120*48, ))
# This is the size of the output. We want to generate 28 x 28 pictures in the end, so this is the size we're looking for. 
output_dim = 120*120*48
encoded1 = Dense(encoding_dim, activation='relu',kernel_regularizer=regularizers.l2(lambda_))(input_img)
decoded1 = Dense(output_dim, activation='sigmoid',kernel_regularizer=regularizers.l2(lambda_))(encoded1)
encoded2 = Dense(encoding_dim, activation='relu',kernel_regularizer=regularizers.l2(lambda_))(decoded1)
decoded2 = Dense(output_dim, activation='sigmoid',kernel_regularizer=regularizers.l2(lambda_))(encoded2)
encoded3 = Dense(encoding_dim, activation='relu',kernel_regularizer=regularizers.l2(lambda_))(decoded2)
decoded3 = Dense(output_dim, activation='sigmoid',kernel_regularizer=regularizers.l2(lambda_))(encoded3)
encoded4 = Dense(encoding_dim, activation='relu',kernel_regularizer=regularizers.l2(lambda_))(decoded3)
decoded4 = Dense(output_dim, activation='sigmoid',kernel_regularizer=regularizers.l2(lambda_))(encoded4)
output = Dense(150,activation='softmax',kernel_regularizer=regularizers.l2(lambda_))(decoded4)

In [0]:
autoencoder = Model(input_img, output)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics=['accuracy'])
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 691200)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                24883236  
_________________________________________________________________
dense_2 (Dense)              (None, 691200)            25574400  
_________________________________________________________________
dense_3 (Dense)              (None, 150)               103680150 
Total params: 154,137,786
Trainable params: 154,137,786
Non-trainable params: 0
_________________________________________________________________


In [0]:
autoencoder.fit(X_train,Y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/1
750/750 [==============================] - 8s 10ms/step - loss: 3.1607


In [0]:
out = autoencoder.predict(X_test)